# Simple example

Define a data format, load cycling data from a file, and do some basic analysis

In [ ]:
import bdat
import pandas as pd
import altair as alt
import numpy as np

Specify the relevant columns in the data file and build some basic metadata, then load the data file.

In [ ]:
# specify columns
dataspec = bdat.DataSpec(
    "neware",
    durationColumn=bdat.TimeColumnSpec("Time##T_+01_00", bdat.Timestamp()),
    currentColumn=bdat.ColumnSpec("Current#A#D"),
    voltageColumn=bdat.ColumnSpec("Voltage#V#D"),
    chargeSpec=bdat.SeparateColumns(bdat.ColumnSpec("Charge Capacity#Ah#D"), bdat.ColumnSpec("Discharge Capacity#Ah#D")),
)

# build metadata
batterytype = bdat.BatterySpecies("Samsung 35E", capacity=3.5, endOfChargeVoltage=4.2, endOfDischargeVoltage=2.7)
battery = bdat.Battery("Samsung_35E_0001", type=batterytype)
cycling = bdat.Cycling("cycling example", object=battery)

# load cycling data from file
data = bdat.CyclingData(cycling, pd.read_parquet("1482756.parquet"), dataspec)

Segment the data into steps based on the current and voltage values. This example uses the default settings used for segmentation, additional options can be found in the documentation.

In [ ]:
# find steps
steplist = bdat.steps.find_steps(data)

# plot steps
bdat.plots.plot_steps(None, steplist, data.df).show()

Look for relevant sequences of steps in the data.

In [ ]:
## find patterns in the test steps

# specify patterns to look for
patterntypes = [
    bdat.patterns.Captest(),
    bdat.patterns.Testinfo(),
    bdat.patterns.UniformCycling(),
]

# search the test steps for matches
testeval = bdat.patterns.find_patterns(steplist, patterntypes, data)

# plot results
patternplot = bdat.plots.plot_testevals(None, testeval, data.df)
alt.Chart.from_dict(patternplot.plot)

Print some information about the patterns that were found.

In [ ]:
for eval in testeval.evals:
    if isinstance(eval, bdat.entities.patterns.UniformCyclingEval):
        print(f"Cycling:\n{eval.cyclecount} cycles, {eval.dod:.2f}% DOD (min. {eval.minSoc:.2f}% SOC, max. {eval.maxSoc:.2f}% SOC)")
    elif isinstance(eval, bdat.entities.patterns.DischargeCapacityEval):
        print(f"Capacity test (step {eval.firstStep}):\n{np.abs(eval.capacity)} Ah")
    if isinstance(eval, bdat.entities.patterns.TestinfoEval):
        print(f"Test info:\n{eval.totalStepCount} steps, {eval.chargeAh:.2f} Ah charged, {eval.dischargeAh:.2f} Ah discharged")
